In [1]:
import os
import pandas as pd
import numpy as np

import pyreadr

#import dask_mpi as dm
from dask.distributed import Client, progress
import dask

from datetime import datetime
from joblib import Parallel, delayed
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load
import joblib

In [2]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
DP02_location = "/Users/lichao/Library/CloudStorage/OneDrive-KyushuUniversity/02_Article/03_RStudio/"
DP02_result_location = "/Users/lichao/Library/CloudStorage/OneDrive-KyushuUniversity/02_Article/03_RStudio/08_PyResults/"

In [2]:
DP02_location = "D:/OneDrive - Kyushu University/02_Article/03_RStudio/"
DP02_result_location = "D:/OneDrive - Kyushu University/02_Article/03_RStudio/08_PyResults/"

In [125]:
client = Client(n_workers=2, nthreads=1)

/Users/lichao/opt/anaconda3/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56738 instead
  warnings.warn(


In [126]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:56738/status,
Dashboard: http://127.0.0.1:56738/status,Workers: 2
Total threads: 10,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56739,Workers: 2
Dashboard: http://127.0.0.1:56738/status,Total threads: 10
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:56747,Total threads: 5
Dashboard: http://127.0.0.1:56749/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:56743,


In [5]:
dataset = pyreadr.read_r(DP02_location + "02_Data/SP_Data_49Variable_Weights_changeRangeOfLandCover_RdsVer.Rds")
dataset = dataset[None]
y = np.array(dataset.iloc[:, 0:1].values.flatten(), dtype='float64')
X = np.array(dataset.iloc[:, 1:50], dtype='float64')

In [6]:
model = RandomForestRegressor(n_estimators=1000, oob_score=True, 
                               random_state=1, max_features = 9, n_jobs=-1)
with joblib.parallel_backend("dask"): model.fit(X, y)

In [7]:
# SHAP
import dalex as dx
model_rf_exp = dx.Explainer(model, X, y, label = "RF Pipeline")

Preparation of a new explainer is initiated

  -> data              : numpy.ndarray converted to pandas.DataFrame. Columns are set as string numbers.
  -> data              : 89273 rows 49 cols
  -> target variable   : 89273 values
  -> model_class       : sklearn.ensemble._forest.RandomForestRegressor (default)
  -> label             : RF Pipeline
  -> predict function  : <function yhat_default at 0x000001C722A12680> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.


D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


  -> predicted values  : min = 1.18, mean = 24.3, max = 36.0
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)


D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


  -> residuals         : min = -11.2, mean = 0.0166, max = 7.13
  -> model_info        : package sklearn

A new explainer has been created!


In [127]:
X_scattered = client.scatter(X)
model_rf_exp_scattered = client.scatter(model_rf_exp)

In [46]:
def singleSHAPprocess(obs_num, X, model_rf_exp):
    test_obs = X[obs_num:obs_num+1,:]
    shap_test = model_rf_exp.predict_parts(test_obs, type = 'shap', 
                                           B = 20, N = 1000)
    result = shap_test.result[shap_test.result.B == 0]
    result = result[['contribution', 'variable_name']]
    result = result.transpose()
    result = result.rename(columns=result.iloc[1])
    result = result.drop(['variable_name'], axis=0)
    result = result.reset_index(drop=True)
    return result

In [47]:
%%time
results = []
for obs_num in list(range(10000)):
    results.append(dask.delayed(singleSHAPprocess)(obs_num, X_scattered, model_rf_exp_scattered))

Wall time: 684 ms


In [ ]:
%%time
start = datetime.now()
test = dask.compute(*results) 
end = datetime.now()

2022-11-13 10:50:39,566 - bokeh.core.property.validation - ERROR - 'str' object has no attribute 'text'
Traceback (most recent call last):
  File "D:\Anaconda3\envs\Usegdal\lib\site-packages\distributed\utils.py", line 767, in wrapper
    return func(*args, **kwargs)
  File "D:\Anaconda3\envs\Usegdal\lib\site-packages\distributed\dashboard\components\scheduler.py", line 355, in update
    self.root.title.text = title
AttributeError: 'str' object has no attribute 'text'
2022-11-13 10:50:39,571 - bokeh.application.handlers.function - ERROR - 'str' object has no attribute 'text'
Traceback (most recent call last):
  File "D:\Anaconda3\envs\Usegdal\lib\site-packages\distributed\utils.py", line 767, in wrapper
    return func(*args, **kwargs)
  File "D:\Anaconda3\envs\Usegdal\lib\site-packages\distributed\dashboard\components\scheduler.py", line 3915, in status_doc
    cluster_memory.update()
  File "D:\Anaconda3\envs\Usegdal\lib\site-packages\bokeh\core\property\validation.py", line 95, in 

In [45]:
%%time
test[999]

Wall time: 0 ns


,18,14,30,10,12,13,23,36,15,48,...,27,6,8,7,5,21,26,4,20,34
0,-0.901555,0.528859,-0.500492,0.484137,-0.402003,0.390392,0.345219,-0.219081,0.185294,-0.11907,...,0.002924,-0.002518,0.00205,0.00163,0.001456,0.00125,0.00055,0.000494,0.0001,0.0


In [39]:
results

[Delayed('singleSHAPprocess-e0910916-34e4-48ec-912b-08e0b00bd092'),
 Delayed('singleSHAPprocess-ff91032e-7a4b-4c2f-92c3-dc5030e54bd6'),
 Delayed('singleSHAPprocess-3dc74a2d-ce04-476c-befb-63723b61fc0c'),
 Delayed('singleSHAPprocess-6ed950fd-5ebd-485c-af84-5f798cfff8b4'),
 Delayed('singleSHAPprocess-c986087a-a481-4778-9084-7731809c26d3'),
 Delayed('singleSHAPprocess-97e50648-10e7-454a-b030-1f936b55fad7'),
 Delayed('singleSHAPprocess-3f9f8b5e-2f28-4f83-a5ec-95fd979ba90b'),
 Delayed('singleSHAPprocess-528e5754-f91c-4484-80de-3dddc0b24488'),
 Delayed('singleSHAPprocess-02c5da84-867c-4a8d-bde0-84085c9bf3ee'),
 Delayed('singleSHAPprocess-bb2e362f-c380-44c1-97e1-b196fa7a1f26'),
 Delayed('singleSHAPprocess-b0e8d820-5b3f-4920-b33f-59f29fc65afb'),
 Delayed('singleSHAPprocess-9d8596bc-791a-45a5-8e63-81c2982b8dd1'),
 Delayed('singleSHAPprocess-4685be39-700a-4854-bb4b-061edaaee009'),
 Delayed('singleSHAPprocess-364e2d01-0c38-4fb9-85ca-5cb5754aed5b'),
 Delayed('singleSHAPprocess-ad63becd-72a5-4307-b

In [71]:
list(range(8))

[0, 1, 2, 3, 4, 5, 6, 7]

In [23]:
client.close()

In [25]:
%%time
results_bag = joblib.Parallel(n_jobs=3, verbose=10000, backend = 'threading')(
    joblib.delayed(singleSHAPprocess)(int(obs_num), X, model_rf_exp)
    for obs_num in list(range(20)))

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

...

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 

D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
D:\Anaconda3\envs\Usegdal\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor 